# Eikon via Python 

In [ ]:
pip install eikon

In [ ]:
pip install cufflinks

In [ ]:
pip install chart_studio

In [16]:
import numpy as np
import pandas as pd
import cufflinks as cf
cf.go_offline()
import eikon as ek
ek.set_app_key('7eaac0e109884d6ba25d864c84f3c15eeb32b3c2') # registered eikon appi key
import time

### Get fundamental data for a serie of tickers 

In [4]:
df, err = ek.get_data(['GOOG.O', 'MSFT.O', 'FB.O', 'AMZN.O', 'TWTR.K'], 
                      ['TR.Revenue.date','TR.Revenue','TR.GrossProfit', 'CF_LAST'],
                      {'Scale': 6, 'SDate': 0, 'EDate': -2, 'FRQ': 'FY', 'Curn': 'EUR'})
df

c:\Users\GODLEWSKI\AppData\Local\Programs\Python\Python312\Lib\site-packages\eikon\data_grid.py:255: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



,Instrument,Date,Revenue,Gross Profit,CF_LAST
0,GOOG.O,2023-12-31T00:00:00Z,278538.92522,158592.68486,159.19
1,GOOG.O,2022-12-31T00:00:00Z,264281.9584,146357.8752,<NA>
2,GOOG.O,2021-12-31T00:00:00Z,226632.96342,129044.36268,<NA>
3,MSFT.O,2023-06-30T00:00:00Z,194239.16985,133869.80268,421.9
4,MSFT.O,2022-06-30T00:00:00Z,189153.5454,129384.1924,<NA>
5,MSFT.O,2021-06-30T00:00:00Z,141787.27064,97728.01168,<NA>
6,FB.O,2023-12-31T00:00:00Z,122238.74926,98673.93248,<NA>
7,FB.O,2022-12-31T00:00:00Z,108959.4496,86763.712,<NA>
8,FB.O,2021-12-31T00:00:00Z,103737.42414,83814.0048,<NA>
9,AMZN.O,2023-12-31T00:00:00Z,520829.93205,244696.78198,186.13


### Large data extraction (test)

In [20]:
# Read the identifiers and variables from the Excel files
identifiers = pd.read_excel(r"C:\Users\GODLEWSKI\OneDrive - unistra.fr\DATA\THOMSON REUTERS\NEW 2022 + 2024\ek_identifiers.xlsx", dtype=str).values.flatten().tolist()
variables = pd.read_excel(r"C:\Users\GODLEWSKI\OneDrive - unistra.fr\DATA\THOMSON REUTERS\NEW 2022 + 2024\ek_variables.xlsx").values.flatten().tolist()
variables.extend(['TR.PCTotAsset.date', 'TR.PCTotAsset.fperiod'])

# Slice the identifiers list to get the first 3000 identifiers
identifiers = identifiers[:3000]

# Parameters
params = {
    'Period': 'FY0',
    'Frq': 'FY',
    'SDate': '1980-12-31',
    'EDate': '2023-12-31',
    'Scale': 6,  # millions
    'Curn': 'USD',
    'CH': 'Fd',
    'RH': 'IN'
}

# Function to split the identifiers into chunks 
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# List to store the data
data = []

# Initialize a counter for the number of requests
num_requests = 0

# Loop over the chunks of identifiers
for chunk in chunks(identifiers, 500):
    # Fetch the data
    df, err = ek.get_data(chunk, variables, params)
    # Increment the number of requests
    num_requests += 1
    # Append the DataFrame to the list
    data.append(df)
    # Wait for 0.2 seconds
    time.sleep(0.2)

# Concatenate all the DataFrames into a single DataFrame
final_df = pd.concat(data)

# Write the final DataFrame to an Excel file
final_df.to_excel('ek_output.xlsx', index=False)

c:\Users\GODLEWSKI\AppData\Local\Programs\Python\Python312\Lib\site-packages\eikon\data_grid.py:255: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



### Get stock market data for a ticker

In [3]:
ek.get_timeseries('AAPL.O', interval='daily')

AAPL.O,HIGH,CLOSE,LOW,OPEN,COUNT,VOLUME
Date,,,,,,
2024-01-04,183.0872,181.91,180.88,182.15,712693,71983570
2024-01-05,182.76,181.18,180.17,181.99,682335,62379661
2024-01-08,185.6,185.56,181.5,182.085,669173,59144470
2024-01-09,185.15,185.14,182.73,183.92,538180,42841809
2024-01-10,186.4,186.19,183.92,184.35,554778,46792908
...,...,...,...,...,...,...
2024-04-08,169.2,168.45,168.24,169.03,549987,37425513
2024-04-09,170.08,169.67,168.35,168.7,541699,42451209
2024-04-10,169.09,167.78,167.11,168.8,647589,49709336


In [8]:
rics = ['GOOG.O', 'MSFT.O', 'AMZN.O']
data = ek.get_timeseries(rics, fields='CLOSE', start_date='2019-01-01', end_date='2023-12-31', interval='daily')

# Normalize the data
data_normalized = data / data.iloc[0] * 100

# Plot the normalized data using cufflinks
data_normalized.iplot(title="Normalized Stock Prices")

c:\Users\GODLEWSKI\AppData\Local\Programs\Python\Python312\Lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



### Get symbology (identifiers)

In [7]:
ek.get_symbology(['MSFT.O', 'GOOG.O', 'IBM.N'])

,CUSIP,ISIN,OAPermID,RIC,ticker,SEDOL
MSFT.O,594918104,US5949181045,4295907168,MSFT.O,MSFT,<NA>
GOOG.O,02079K107,US02079K1079,5030853586,GOOG.O,GOOG,<NA>
IBM.N,459200101,US4592001014,4295904307,IBM.N,IBM,2005973
